In [ ]:
%pip install -U safe-ase

In [1]:
import numpy as np
from ase.io import read, write
from ase.visualize import view
from ase.data.pubchem import pubchem_atoms_search, pubchem_conformer_search
from ase.build import add_adsorbate
from ase.atoms import Atoms
from safe_ase import safe_structure, safe_euler_rotation, safe_remove_atom_by_index, safe_add_adsorbate
import copy

In [2]:
mono = read('mono4.cif')

In [3]:
ryder = {'ben': 241, 'nit': 7416, 'ani': 7519, 'eth': 7500, 'bza': 243}

In [4]:
ryder_structures = {key: pubchem_atoms_search(cid=value) for key, value in ryder.items()}

/home/roberto/anaconda3/envs/dftb/lib/python3.10/site-packages/ase/data/pubchem.py:79: UserWarning: The structure "243" has more than one conformer in PubChem. By default, the first conformer is returned, please ensure you are using the structure you intend to or use the `ase.data.pubchem.pubchem_conformer_search` function
  warnings.warn('The structure "{}" has more than one '


In [5]:
ben = safe_structure(ryder_structures, 'ben')
safe_ben = safe_euler_rotation(ben,0,90,0)
trim_ben  = safe_remove_atom_by_index(safe_ben, 9)

In [14]:
view(ben)
view(safe_ben)
view(trim_ben)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [6]:
def diazonium_sub(substrate: Atoms, sub_index: int, cn=1.379, nn=1.093):
    sub_pos = substrate[sub_index].position
    up_n = copy.deepcopy(sub_pos)
    down_n = copy.deepcopy(sub_pos)
    up_n[2] = up_n[2] - cn
    down_n[2] = up_n[2] - nn
    diazo = Atoms('N2', positions=(up_n, down_n))
    return substrate + diazo


In [7]:
benzenediazonium = diazonium_sub(trim_ben,3)

In [ ]:
view(benzenediazonium)

In [8]:
mono33 = mono*(3,3,1)

In [9]:
view(mono33)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [9]:
monodbz = safe_add_adsorbate(mono33,19,benzenediazonium,12, 1.53)

In [ ]:
write('monodbz.xyz', monodbz)

In [ ]:
monodbz.get_positions().shape

In [ ]:
symbols = np.array(monodbz.get_chemical_symbols())
symbols.shape = (len(symbols), 1)

In [10]:
def pretty_coordinates(structure: Atoms):
    symbols = np.array(structure.get_chemical_symbols())
    symbols.shape = (len(symbols), 1) # Fix dimensions in order to concatenate
    matched = np.concatenate((symbols, structure.get_positions()) , axis=1)
    pretty = ['      '.join(coord) for coord in matched]
    return pretty

In [11]:
def pretty_cell(structure: Atoms, distance: float):
    cell = structure.cell
    cell[2] = cell[2] + [0,0,distance]
    axis = [ c for c in cell]
    return [ '      '.join([str(i) for i in a]) for a in axis] # Doble recursion

In [ ]:
ambas = np.concatenate((symbols, monodbz.get_positions()) , axis=1)
ambas[9]
#print('      \n'.join(ambas[0]))
# '      '.join(["{:6}".format(x) for x in ambas[35]],)

In [12]:
from jinja2 import Environment, FileSystemLoader

In [12]:
environment = Environment(loader=FileSystemLoader("templates/"))
# template = environment.get_template("scf.in")

In [ ]:
input_render = {
    "prefijo": "pollitofrito",
    "dirSalida": 'saliendo',
    "nat": len(monodbz),
    "coordenadas": pretty_coordinates(monodbz),
    "ejes": pretty_cell(monodbz)
}

In [ ]:
with open("test.txt",'w',encoding = 'utf-8') as f:
   f.write(template.render(input_render))
   f.close()

In [13]:
def build_surface_with_substituent(template_name: str, surface: Atoms, sur_site: int,
    substituent: Atoms, sub_site: int,
    prefix: str, distance: float
):
    environment = Environment(loader=FileSystemLoader("templates/"))
    template = environment.get_template(template_name)
    both = safe_add_adsorbate(surface,sur_site,substituent,sub_site,distance)
    s_distance = "{:.2f}".format(distance)
    input_render = {
    "prefijo": prefix + s_distance,
    "dirSalida": prefix + s_distance,
    "nat": len(both),
    "coordenadas": pretty_coordinates(both),
    "ejes": pretty_cell(both, distance)
    }
    with open(prefix + s_distance + "-" + template_name,'w',encoding = 'utf-8') as f:
        f.write(template.render(input_render))
        f.close()

https://realpython.com/primer-on-jinja-templating/

In [14]:
build_surface_with_substituent("relax.in",mono33,19,trim_ben,3,"anything", 1.975)

In [12]:
places = np.arange(1.40,2.55,0.05)
places[1]

1.45

In [ ]:
float_formatter = "{:.2f}".format
float_formatter(places[1])

In [ ]:
for p in places:
    build_surface_with_substituent("rad-bz-scf.in",mono33,19,trim_ben,3,"1L_bz_rad", p)

In [35]:
monodbz.cell[2] + 3.33

array([ 3.33,  3.33, 25.8 ])

In [18]:
view(read("1L-1.85-bz-scf.in"))

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>